In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from itertools import combinations
warnings.filterwarnings("ignore")

In [2]:
file_path = 'train.csv'  
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,id,stmt_date,HQ_city,legal_struct,ateco_sector,def_date,fs_year,asst_intang_fixed,asst_tang_fixed,...,inc_extraord,taxes,profit,days_rec,ebitda,roa,roe,wc_net,margin_fin,cf_operations
0,17,520288,2011-12-31,28.0,SRL,14.0,NaT,2011,67537.0,1137566.0,...,-309.0,-80959.0,-81288.0,NaN,6318.0,-3.81,-28.03,496258.0,-917029.0,-849.0
1,18,520288,2008-12-31,28.0,SRL,14.0,NaT,2008,256438.0,1181416.0,...,-678.0,-94622.0,-107382.0,NaN,46088.0,-2.76,NaN,97952.0,NaN,-3881.0
2,19,520288,2009-12-31,28.0,SRL,14.0,NaT,2009,194046.0,1152014.0,...,4224.0,-74235.0,-77819.0,NaN,67611.0,-2.17,NaN,-210671.0,NaN,32618.0
3,21,520288,2012-12-31,28.0,SRL,14.0,NaT,2012,15195.0,1116938.0,...,3634.0,-250786.0,-250786.0,NaN,-161478.0,-12.99,NaN,367892.0,-1094962.0,-168907.0
4,22,520288,2007-12-31,28.0,SRL,14.0,NaT,2007,126603.0,1127807.0,...,820.0,92192.0,36733.0,NaN,153060.0,6.20,52.43,-317007.0,-1184970.0,80039.0


In [5]:
import pandas as pd
import numpy as np
from itertools import combinations
from joblib import Parallel, delayed
from tqdm import tqdm  # Progress bar for tracking

def find_summation_relationships(df, tolerance=0.2, sample_size=10000, max_comb_size=4):
    """
    Finds columns in `df` that are summations of other columns within a given tolerance.
    
    Parameters:
    - df: DataFrame to analyze
    - tolerance: Allowed tolerance for summation checks
    - sample_size: Number of rows to sample for faster computation (optional)
    - max_comb_size: Maximum combination size of columns to check for summations
    
    Returns:
    - Dictionary with target columns as keys and tuples of column combinations that sum to them as values.
    """
    # Use only numeric columns
    numeric_df = df.select_dtypes(include=[np.number])

    # Apply sampling for faster computation
    if len(numeric_df) > sample_size:
        numeric_df = numeric_df.sample(sample_size, random_state=42)
    
    results = {}

    def check_summation(target_col, other_cols, max_comb_size, tolerance):
        target_series = numeric_df[target_col].fillna(0)
        for num_cols in range(2, max_comb_size + 1):  # Limit combination size
            for subset in combinations(other_cols, num_cols):
                subset_sum = numeric_df[list(subset)].sum(axis=1).fillna(0)
                difference = (target_series - subset_sum).abs()
                
                # If all differences are within the tolerance, log the result
                if (difference <= tolerance).all():
                    return (target_col, subset)
        return None

    # Parallel processing for each target column
    other_cols_dict = {col: [c for c in numeric_df.columns if c != col] for col in numeric_df.columns}
    results_list = Parallel(n_jobs=-1)(
        delayed(check_summation)(col, other_cols_dict[col], max_comb_size, tolerance) 
        for col in tqdm(numeric_df.columns)
    )
    
    # Filter out None results
    for res in results_list:
        if res:
            results[res[0]] = res[1]
    
    return results

# Example usage:
file_path = 'train.csv'  
df = pd.read_csv(file_path)
summation_relationships = find_summation_relationships(df)
print(summation_relationships)


100%|███████████████████████████████████████████| 41/41 [01:20<00:00,  1.96s/it]


{}
